<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>3D Volume Rendering with LOD and Raycasting</title>
    <style>
        body, html {
            margin: 0;
            padding: 0;
            height: 100%;
            overflow: hidden;
        }
        #webglCanvas {
            width: 100%;
            height: 100%;
            display: block;
        }
        .gui-container {
            position: absolute;
            top: 0;
            right: 0;
            z-index: 100;
        }
    </style>
    <!-- dat.GUI ライブラリを利用するためのスクリプト -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/dat-gui/0.7.7/dat.gui.min.js"></script>
</head>
<body>
    <!-- WebGL2 を表示するキャンバス -->
    <canvas id="webglCanvas"></canvas>

    <!-- dat.GUI コントロールを表示するコンテナ -->
    <div class="gui-container"></div>

    <script>
        const canvas = document.getElementById('webglCanvas');
        const gl = canvas.getContext('webgl2');

        if (!gl) {
            alert('WebGL2 がサポートされていません');
        }

        // 頂点シェーダーとフラグメントシェーダーのソースコード
        const vertexShaderSource = `#version 300 es
        precision highp float;
        in vec3 aPosition;
        uniform mat4 uModelViewMatrix;
        uniform mat4 uProjectionMatrix;
        void main() {
            gl_Position = uProjectionMatrix * uModelViewMatrix * vec4(aPosition, 1.0);
        }`;

        const fragmentShaderSource = `#version 300 es
        precision highp float;
        out vec4 fragColor;
        uniform sampler3D uLowResTexture;
        uniform sampler3D uMidResTexture;
        uniform sampler3D uHighResTexture;
        uniform float uLOD;
        uniform float uGamma;
        uniform float uBrightness;
        uniform float uContrast;
        uniform int uChannel; // RGB チャネル選択
        in vec3 vTexCoord;
        void main() {
            vec4 color;
            if (uLOD < 1.0) {
                color = texture(uLowResTexture, vTexCoord);
            } else if (uLOD < 2.0) {
                color = texture(uMidResTexture, vTexCoord);
            } else {
                color = texture(uHighResTexture, vTexCoord);
            }

            if (uChannel == 0) {
                color = vec4(color.r, color.r, color.r, 1.0);
            } else if (uChannel == 1) {
                color = vec4(color.g, color.g, color.g, 1.0);
            } else {
                color = vec4(color.b, color.b, color.b, 1.0);
            }

            color.rgb = pow(color.rgb, vec3(uGamma));
            color.rgb = (color.rgb - 0.5) * uContrast + 0.5;
            color.rgb += uBrightness;

            fragColor = color;
        }`;

        function createShader(gl, type, source) {
            const shader = gl.createShader(type);
            gl.shaderSource(shader, source);
            gl.compileShader(shader);
            if (!gl.getShaderParameter(shader, gl.COMPILE_STATUS)) {
                console.error('シェーダーのコンパイルエラー:', gl.getShaderInfoLog(shader));
                gl.deleteShader(shader);
                return null;
            }
            return shader;
        }

        const vertexShader = createShader(gl, gl.VERTEX_SHADER, vertexShaderSource);
        const fragmentShader = createShader(gl, gl.FRAGMENT_SHADER, fragmentShaderSource);

        const program = gl.createProgram();
        gl.attachShader(program, vertexShader);
        gl.attachShader(program, fragmentShader);
        gl.linkProgram(program);

        const lowResTexture = gl.createTexture();
        const midResTexture = gl.createTexture();
        const highResTexture = gl.createTexture();

        function setupTexture(texture, data, width, height, depth) {
            gl.bindTexture(gl.TEXTURE_3D, texture);
            gl.texImage3D(gl.TEXTURE_3D, 0, gl.RGBA, width, height, depth, 0, gl.RGBA, gl.UNSIGNED_BYTE, data);
            gl.texParameteri(gl.TEXTURE_3D, gl.TEXTURE_MIN_FILTER, gl.LINEAR);
            gl.texParameteri(gl.TEXTURE_3D, gl.TEXTURE_MAG_FILTER, gl.LINEAR);
        }

        async function loadVolumeData(url) {
            const response = await fetch(url);
            const arrayBuffer = await response.arrayBuffer();
            return new Uint8Array(arrayBuffer);
        }

        async function loadTextures() {
            const lowResData = await loadVolumeData('low_res_data_url');
            const midResData = await loadVolumeData('mid_res_data_url');
            const highResData = await loadVolumeData('high_res_data_url');

            setupTexture(lowResTexture, lowResData, 512, 512, 100);
            setupTexture(midResTexture, midResData, 1024, 1024, 200);
            setupTexture(highResTexture, highResData, 2048, 2048, 400);
        }

        loadTextures();

        const uLowResTextureLocation = gl.getUniformLocation(program, 'uLowResTexture');
        const uMidResTextureLocation = gl.getUniformLocation(program, 'uMidResTexture');
        const uHighResTextureLocation = gl.getUniformLocation(program, 'uHighResTexture');
        const uLODLocation = gl.getUniformLocation(program, 'uLOD');
        const uGammaLocation = gl.getUniformLocation(program, 'uGamma');
        const uBrightnessLocation = gl.getUniformLocation(program, 'uBrightness');
        const uContrastLocation = gl.getUniformLocation(program, 'uContrast');
        const uChannelLocation = gl.getUniformLocation(program, 'uChannel');

        let isDragging = false;
        let previousMousePosition = { x: 0, y: 0 };
        let rotation = [0, 0];
        let cameraDistance = 5.0;

        function calculateLOD(cameraDistance) {
            if (cameraDistance > 10.0) {
                return 0.0;
            } else if (cameraDistance > 5.0) {
                return 1.0;
            } else {
                return 2.0;
            }
        }

        canvas.addEventListener('mousedown', function(event) {
            isDragging = true;
            previousMousePosition = { x: event.clientX, y: event.clientY };
        });

        canvas.addEventListener('mouseup', function() {
            isDragging = false;
        });

        canvas.addEventListener('mousemove', function(event) {
            if (isDragging) {
                const deltaX = event.clientX - previousMousePosition.x;
                const deltaY = event.clientY - previousMousePosition.y;
                rotation[0] += deltaX * 0.01;
                rotation[1] += deltaY * 0.01;
                previousMousePosition = { x: event.clientX, y: event.clientY };
            }
        });

        canvas.addEventListener('wheel', function(event) {
            const zoomSpeed = 0.1;
            cameraDistance += event.deltaY * zoomSpeed;
            cameraDistance = Math.max(1, cameraDistance);
        });

        const gui = new dat.GUI();
        let gamma = 2.2;
        let brightness = 0.0;
        let contrast = 1.0;
        let channel = 0;
        gui.add({ gamma: gamma }, 'gamma', 0.1, 5).onChange(function(value) {
            gamma = value;
        });
        gui.add({ brightness: brightness }, 'brightness', -1, 1).onChange(function(value) {
            brightness = value;
        });
        gui.add({ contrast: contrast }, 'contrast', 0.1, 2).onChange(function(value) {
            contrast = value;
        });
        gui.add({ channel: channel }, 'channel', { R: 0, G: 1, B: 2 }).onChange(function(value) {
            channel = value;
        });

        function render() {
            gl.clear(gl.COLOR_BUFFER_BIT | gl.DEPTH_BUFFER_BIT);
            gl.useProgram(program);

            const lod = calculateLOD(cameraDistance);
            gl.uniform1f(uLODLocation, lod);
            gl.uniform1f(uGammaLocation, gamma);
            gl.uniform1f(uBrightnessLocation, brightness);
            gl.uniform1f(uContrastLocation, contrast);
            gl.uniform1i(uChannelLocation, channel);

            gl.bindTexture(gl.TEXTURE_3D, lod < 1.0 ? lowResTexture : lod < 2.0 ? midResTexture : highResTexture);

            requestAnimationFrame(render);

In [ ]:
// 캔버스 가져오기
const canvas = document.getElementById('webglCanvas');
const gl = canvas.getContext('webgl2');

// 셰이더 소스 코드 (레이 캐스팅용)
const vertexShaderSource = `#version 300 es
precision highp float;
in vec3 aPosition;
uniform mat4 uModelViewMatrix;
uniform mat4 uProjectionMatrix;
void main() {
    gl_Position = uProjectionMatrix * uModelViewMatrix * vec4(aPosition, 1.0);
}`;

const fragmentShaderSource = `#version 300 es
precision highp float;
out vec4 fragColor;
uniform sampler3D uTexture;
uniform vec3 uCameraPosition;
uniform vec3 uLightPosition;
uniform float uGamma;
uniform float uBrightness;
uniform float uContrast;
uniform int uChannel; // RGB 채널 선택
in vec3 vTexCoord;
void main() {
    // 레이 캐스팅 알고리즘
    vec3 rayDir = normalize(vTexCoord - uCameraPosition);
    vec3 rayPos = uCameraPosition;

    // RGB 채널 선택 (uChannel: 0=R, 1=G, 2=B)
    vec4 color;
    if (uChannel == 0) {
        color = texture(uTexture, rayPos).rrrr;
    } else if (uChannel == 1) {
        color = texture(uTexture, rayPos).gggg;
    } else {
        color = texture(uTexture, rayPos).bbbb;
    }

    // 대비 및 감마 보정
    color.rgb = pow(color.rgb, vec3(uGamma));
    color.rgb = mix(vec3(0.5), mix(vec3(0.0), vec3(1.0), color.rgb) * uContrast, 0.5);

    // 밝기 조정
    color.rgb += uBrightness;

    fragColor = color;
}`;

// 셰이더 컴파일 및 링크 함수
function createShader(gl, type, source) {
    const shader = gl.createShader(type);
    gl.shaderSource(shader, source);
    gl.compileShader(shader);
    if (!gl.getShaderParameter(shader, gl.COMPILE_STATUS)) {
        console.error('셰이더 컴파일 에러:', gl.getShaderInfoLog(shader));
        gl.deleteShader(shader);
        return null;
    }
    return shader;
}

// 셰이더 프로그램 생성
const vertexShader = createShader(gl, gl.VERTEX_SHADER, vertexShaderSource);
const fragmentShader = createShader(gl, gl.FRAGMENT_SHADER, fragmentShaderSource);

const program = gl.createProgram();
gl.attachShader(program, vertexShader);
gl.attachShader(program, fragmentShader);
gl.linkProgram(program);

// 3D 텍스처 생성 및 설정 (Material용)
const volumeTexture = gl.createTexture();
gl.bindTexture(gl.TEXTURE_3D, volumeTexture);
gl.texParameteri(gl.TEXTURE_3D, gl.TEXTURE_MIN_FILTER, gl.LINEAR);
gl.texParameteri(gl.TEXTURE_3D, gl.TEXTURE_MAG_FILTER, gl.LINEAR);
gl.texParameteri(gl.TEXTURE_3D, gl.TEXTURE_WRAP_S, gl.CLAMP_TO_EDGE);
gl.texParameteri(gl.TEXTURE_3D, gl.TEXTURE_WRAP_T, gl.CLAMP_TO_EDGE);
gl.texParameteri(gl.TEXTURE_3D, gl.TEXTURE_WRAP_R, gl.CLAMP_TO_EDGE);

// 텍스처 데이터 설정 (volumeData는 적절히 초기화되어야 함)
gl.texImage3D(gl.TEXTURE_3D, 0, gl.RGBA, 2048, 2048, 400, 0, gl.RGBA, gl.UNSIGNED_BYTE, volumeData);

// 카메라 및 조명, 파라미터 초기값
let cameraPosition = [0, 0, 5];
let lightPosition = [10, 10, 10];
let gamma = 2.2;
let brightness = 0.0;
let contrast = 1.0;
let channel = 0; // R 채널 초기화

// 유니폼 위치 가져오기
const uTextureLocation = gl.getUniformLocation(program, 'uTexture');
const uCameraPositionLocation = gl.getUniformLocation(program, 'uCameraPosition');
const uLightPositionLocation = gl.getUniformLocation(program, 'uLightPosition');
const uGammaLocation = gl.getUniformLocation(program, 'uGamma');
const uBrightnessLocation = gl.getUniformLocation(program, 'uBrightness');
const uContrastLocation = gl.getUniformLocation(program, 'uContrast');
const uChannelLocation = gl.getUniformLocation(program, 'uChannel');
const uModelViewMatrixLocation = gl.getUniformLocation(program, 'uModelViewMatrix');
const uProjectionMatrixLocation = gl.getUniformLocation(program, 'uProjectionMatrix');

// 마우스 조작 변수
let isDragging = false;
let previousMousePosition = { x: 0, y: 0 };
let rotation = [0, 0];

// 마우스 드래그로 회전하는 함수
canvas.addEventListener('mousedown', function(event) {
    isDragging = true;
    previousMousePosition = { x: event.clientX, y: event.clientY };
});

canvas.addEventListener('mouseup', function() {
    isDragging = false;
});

canvas.addEventListener('mousemove', function(event) {
    if (isDragging) {
        const deltaX = event.clientX - previousMousePosition.x;
        const deltaY = event.clientY - previousMousePosition.y;
        rotation[0] += deltaX * 0.01;
        rotation[1] += deltaY * 0.01;
        previousMousePosition = { x: event.clientX, y: event.clientY };
    }
});

// 마우스 휠로 줌 인/아웃 구현
canvas.addEventListener('wheel', function(event) {
    const zoomSpeed = 0.1;
    cameraPosition[2] += event.deltaY * zoomSpeed;
    cameraPosition[2] = Math.max(1, cameraPosition[2]); // 줌 인 제한
});

// GUI로 파라미터 조정하기 위한 슬라이더 설정 (예: dat.GUI 라이브러리)
const gui = new dat.GUI();
gui.add({ gamma: gamma }, 'gamma', 0.1, 5).onChange(function(value) {
    gamma = value;
});
gui.add({ brightness: brightness }, 'brightness', -1, 1).onChange(function(value) {
    brightness = value;
});
gui.add({ contrast: contrast }, 'contrast', 0.1, 2).onChange(function(value) {
    contrast = value;
});
gui.add({ channel: channel }, 'channel', { R: 0, G: 1, B: 2 }).onChange(function(value) {
    channel = value;
});

// 렌더링 루프
function render() {
    gl.clear(gl.COLOR_BUFFER_BIT | gl.DEPTH_BUFFER_BIT);
    gl.useProgram(program);

    // 유니폼에 값 전달
    gl.uniform3fv(uCameraPositionLocation, cameraPosition);
    gl.uniform3fv(uLightPositionLocation, lightPosition);
    gl.uniform1f(uGammaLocation, gamma);
    gl.uniform1f(uBrightnessLocation, brightness);
    gl.uniform1f(uContrastLocation, contrast);
    gl.uniform1i(uChannelLocation, channel);

    // 텍스처 바인딩
    gl.activeTexture(gl.TEXTURE0);
    gl.bindTexture(gl.TEXTURE_3D, volumeTexture);
    gl.uniform1i(uTextureLocation, 0);

    // 모델 뷰 행렬 및 프로젝션 행렬 설정
    const modelViewMatrix = calculateModelViewMatrix(rotation, cameraPosition);
    const projectionMatrix = calculateProjectionMatrix(cameraPosition);
    gl.uniformMatrix4fv(uModelViewMatrixLocation, false, modelViewMatrix);
    gl.uniformMatrix4fv(uProjectionMatrixLocation, false, projectionMatrix);

    // 메쉬 그리기
    gl.drawArrays(gl.TRIANGLES, 0, 36);

    requestAnimationFrame(render);
}

// 모델 뷰 행렬 계산 함수
function calculateModelViewMatrix(rotation, cameraPosition) {
    const modelViewMatrix = mat4.create(); // 실제 계산 필요
    mat4.rotateX(modelViewMatrix, modelViewMatrix, rotation[1]);
    mat4.rotateY(modelViewMatrix, modelViewMatrix, rotation[0]);
    mat4.translate(modelViewMatrix, modelViewMatrix, [-cameraPosition[0], -cameraPosition[1], -cameraPosition[2]]);
    return modelViewMatrix;
}

// 프로젝션 행렬 계산 함수
function calculateProjectionMatrix(cameraPosition) {
    const projectionMatrix = mat4.create(); // 실제 계산 필요
    mat4.perspective(projectionMatrix, Math.PI / 4, canvas.width / canvas.height, 0.1, 100);
    return projectionMatrix;
}

// 렌더링 시작
requestAnimationFrame(render);

In [ ]:
from PIL import Image
import os

# 원본 이미지 크기와 타일 크기 설정
original_image_size = (10000, 10000)
tile_size = (2000, 2000)  # 각 타일의 크기
num_tiles_per_image = (original_image_size[0] // tile_size[0]) * (original_image_size[1] // tile_size[1])

# 출력 디렉토리 생성
output_dir = 'tiles'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 원본 이미지를 생성 (여기서는 랜덤 색상으로 생성)
original_image = Image.new('RGB', original_image_size)
for i in range(original_image_size[0]):
    for j in range(original_image_size[1]):
        original_image.putpixel((i, j), (i % 256, j % 256, (i + j) % 256))

# 타일 분할
for img_index in range(50):
    for x in range(0, original_image_size[0], tile_size[0]):
        for y in range(0, original_image_size[1], tile_size[1]):
            # 타일 이미지 추출
            tile = original_image.crop((x, y, x + tile_size[0], y + tile_size[1]))
            tile.save(os.path.join(output_dir, f'tile_{img_index * num_tiles_per_image + (x // tile_size[0]) * (original_image_size[1] // tile_size[1]) + (y // tile_size[1])}.png'))

print(f'{num_tiles_per_image * 50}개의 타일 이미지가 {output_dir}에 생성되었습니다.')

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>Volume Rendering</title>
    <style>
        canvas { width: 100%; height: 100%; display: block; }
    </style>
</head>
<body>
    <canvas id="canvas"></canvas>
    <script src="https://threejs.org/build/three.js"></script>
    <script src="https://threejs.org/examples/js/controls/OrbitControls.js"></script>
    <script>
        const scene = new THREE.Scene();
        const camera = new THREE.PerspectiveCamera(75, window.innerWidth / window.innerHeight, 0.1, 1000);
        const renderer = new THREE.WebGLRenderer({ canvas: document.getElementById('canvas') });
        renderer.setSize(window.innerWidth, window.innerHeight);
        document.body.appendChild(renderer.domElement);

        // 카메라 위치
        camera.position.set(5, 5, 5);
        camera.lookAt(0, 0, 0);

        const controls = new THREE.OrbitControls(camera, renderer.domElement);

        const tileWidth = 1;
        const tileHeight = 1;
        const tileDepth = 1;

        const gridSize = 5; // x, y, z 축의 타일 갯수
        const tiles = []; // 타일 이미지 경로 배열

        // 타일 이미지 로드
        for (let i = 0; i < 50; i++) {
            tiles.push(`path/to/tile_${i}.png`); // 실제 이미지 경로로 수정해야 합니다.
        }

        const textureLoader = new THREE.TextureLoader();

        for (let x = 0; x < gridSize; x++) {
            for (let y = 0; y < gridSize; y++) {
                for (let z = 0; z < gridSize; z++) {
                    const tileIndex = (x + y * gridSize + z * gridSize * gridSize) % tiles.length;

                    textureLoader.load(tiles[tileIndex], (texture) => {
                        const material = new THREE.MeshBasicMaterial({ map: texture });
                        const geometry = new THREE.BoxGeometry(tileWidth, tileHeight, tileDepth);
                        const cube = new THREE.Mesh(geometry, material);

                        // 각 타일의 위치 설정
                        cube.position.set(x * tileWidth, y * tileHeight, z * tileDepth);
                        scene.add(cube);
                    });
                }
            }
        }

        function animate() {
            requestAnimationFrame(animate);
            controls.update(); // 컨트롤 업데이트
            renderer.render(scene, camera);
        }
        animate();
    </script>
</body>
</html>

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Runtime.InteropServices;

public class BitmapInfo
{
    public int Width;
    public int Height;
    public IntPtr PixelData; // Bitmap 이미지 데이터의 포인터
}

public class Program
{
    // List<BitmapInfo>를 IntPtr 배열로 변환하는 함수 (LINQ 적용)
    public static IntPtr ConvertBitmapInfoListToIntPtr(List<BitmapInfo> bitmapInfos)
    {
        // LINQ로 각각의 BitmapInfo 객체를 GCHandle로 고정하여 IntPtr 배열로 변환
        IntPtr[] ptrArray = bitmapInfos
            .Select(bmp => GCHandle.ToIntPtr(GCHandle.Alloc(bmp, GCHandleType.Pinned)))
            .ToArray();

        // IntPtr 배열을 unmanaged 메모리에 복사합니다.
        IntPtr unmanagedArray = Marshal.AllocHGlobal(ptrArray.Length * Marshal.SizeOf(typeof(IntPtr)));
        Marshal.Copy(ptrArray, 0, unmanagedArray, ptrArray.Length);

        return unmanagedArray;
    }

    // 메모리 해제 함수 (LINQ 적용)
    public static void FreeIntPtr(IntPtr ptr, int count)
    {
        IntPtr[] ptrArray = new IntPtr[count];
        Marshal.Copy(ptr, ptrArray, 0, count);

        // LINQ를 사용하여 GCHandle을 해제
        ptrArray
            .Where(p => p != IntPtr.Zero)
            .ToList()
            .ForEach(p => GCHandle.FromIntPtr(p).Free());

        // 배열 자체의 메모리도 해제
        Marshal.FreeHGlobal(ptr);
    }

    public static void Main()
    {
        List<BitmapInfo> bitmapInfoList = new List<BitmapInfo>
        {
            new BitmapInfo { Width = 100, Height = 100, PixelData = IntPtr.Zero },
            new BitmapInfo { Width = 200, Height = 200, PixelData = IntPtr.Zero },
            // 추가적으로 8개 객체 생성
        };

        // List<BitmapInfo>를 IntPtr로 변환
        IntPtr ptr = ConvertBitmapInfoListToIntPtr(bitmapInfoList);

        // 사용 후 메모리 해제
        FreeIntPtr(ptr, bitmapInfoList.Count);
    }
}